## Cropping for the experimental GAN with anti-aliasing

In [1]:
from PIL import Image
import os
import numpy as np
import tensorflow as tf

In [2]:
imdir = "C:/Users/romat/Desktop/somefolder"

## Run with care! The below cell will take a long time to run, so skip it if you have the images ready

In [3]:
number = 0
h = 163
for image in os.listdir(imdir):
  im = Image.open(imdir + "/" + str(image))
  for rotation in range(0, 4):
    im = im.rotate(angle = 90, expand = True)
    imgwidth, imgheight = im.size
    x = int(imgwidth / h)
    y = int(imgheight / h)
    for i in range(0, y):
      for j in np.arange(0, x):
        name = str(number).zfill(6)
        im.crop((j * h, i * h, (j + 1) * h, (i + 1) * h)).save("C:/Users/romat/Desktop/test/" + name + ".png")
        number += 1

In [4]:
oversampling = 2  # Oversampling factor, 2 is a good choice
kernelN = 9  # Horizontal size of the kernel, also its vertical size. Must be odd. 9 is a good choice when oversampling == 2.

from scipy import special

def circularLowpassKernel(omega_c, N):  # omega = cutoff frequency in radians (pi is max), N = horizontal size of the kernel, also its vertical size, must be odd.
  kernel = np.fromfunction(lambda x, y: omega_c*special.j1(omega_c*np.sqrt((x - (N - 1)/2)**2 + (y - (N - 1)/2)**2))/(2*np.pi*np.sqrt((x - (N - 1)/2)**2 + (y - (N - 1)/2)**2)), [N, N])
  kernel[(N - 1)//2, (N - 1)//2] = omega_c**2/(4*np.pi)
  return kernel 

def rotatedCosineWindow(N):  # N = horizontal size of the targeted kernel, also its vertical size, must be odd.
  return np.fromfunction(lambda y, x: np.maximum(np.cos(np.pi/2*np.sqrt(((x - (N - 1)/2)/((N - 1)/2 + 1))**2 + ((y - (N - 1)/2)/((N - 1)/2 + 1))**2)), 0), [N, N])

window = rotatedCosineWindow(kernelN)

kernel_cutoff_pi = circularLowpassKernel(np.pi/oversampling, kernelN)
kernel_cutoff_minipi = circularLowpassKernel(0.95 * np.pi/oversampling, kernelN)

windowedkernel_cutoff_minipi = kernel_cutoff_minipi*window
windowedkernel_cutoff_minipi /= np.sum(windowedkernel_cutoff_minipi, axis=(0, 1)) # Normalize to sum = 1
windowedkernel_cutoff_minipi = windowedkernel_cutoff_minipi.astype(np.float32)


windowedkernel_cutoff_pi = kernel_cutoff_pi*window
windowedkernel_cutoff_pi /= np.sum(windowedkernel_cutoff_pi, axis=(0, 1)) # Normalize to sum = 1
windowedkernel_cutoff_pi = windowedkernel_cutoff_pi.astype(np.float32)

C:\Users\romat\AppData\Local\Temp/ipykernel_444/18809698.py:7: RuntimeWarning: invalid value encountered in true_divide
  kernel = np.fromfunction(lambda x, y: omega_c*special.j1(omega_c*np.sqrt((x - (N - 1)/2)**2 + (y - (N - 1)/2)**2))/(2*np.pi*np.sqrt((x - (N - 1)/2)**2 + (y - (N - 1)/2)**2)), [N, N])


In [5]:
def filter_target(noise, kernel):
  reps = tf.shape(noise)[-1]
  channeled_kernel = tf.repeat(kernel, reps)
  noise = tf.nn.depthwise_conv2d(noise, tf.reshape(channeled_kernel, (kernel.shape[0], kernel.shape[1], reps, 1)), [1, 1, 1, 1], padding = "VALID")
  return noise

In [6]:
number = 0
#h1 = 78
imdir = "C:/Users/romat/Desktop/filtered/"
for image in os.listdir("C:/Users/romat/Desktop/test/"):
  im = Image.open("C:/Users/romat/Desktop/test/" + str(image))
  #im = im.resize((h1, h1))
  im = tf.keras.preprocessing.image.img_to_array(im)
  im = tf.expand_dims(im, axis = 0)
  im = filter_target(im, windowedkernel_cutoff_pi)
  name = str(number).zfill(6)
  tf.keras.preprocessing.image.save_img((imdir + str(name) + ".png"), im[0])
  number += 1